# FABRIC Reports API Access

This code sets up a **Reports API client** for querying FABRIC sliver and project data directly from the [FABRIC Reports service](https://reports.fabric-testbed.net/reports).

Key points:

* It uses the `fabric_reports_client.reports_api.ReportsApi` class to authenticate and send queries.
* Authentication requires a valid **ID token JSON file**.

  * You must update the `token_file` path in the code to point to **your own token file** (replace the hardcoded `/Users/kthare10/work/id_token_prod.json` with the correct path on your system).
  * If running this on JH, 
    * change `token_file` to point to `/home/fabric/.tokens.json`
    * Install fabric_reports_client
        ```
        pip install fabric_reports_client
        ```
* By default, the client is configured to connect to the public FABRIC Reports service at:

  ```
  https://reports.fabric-testbed.net/reports
  ```
* The `query_end` parameter is automatically set to the current UTC timestamp (ISO-8601 with millisecond precision).
* The `fabric_projects` list contains a set of project IDs to **exclude** from queries. This allows you to filter out specific FABRIC projects you don’t want included in the results.
* By default, it also prints the json output, set `DUMP_JSON` to `False` if json output is not required.

With this setup, you can now issue queries to retrieve sliver records, apply filters (time range, sliver state, component type, site, user, etc.), and explore results in Pandas DataFrames.


In [19]:
from fabric_reports_client.reports_api import ReportsApi
from datetime import datetime, timezone
import pandas as pd
import json

token_file="/Users/kthare10/work/id_token_prod.json"
# Uncomment if running this on JH
#token_file="/home/fabric/.tokens.json"
api =  ReportsApi(base_url="https://reports.fabric-testbed.net/reports", token_file=token_file)

query_start=None

query_end = datetime.now(timezone.utc)
query_end = query_end.isoformat(timespec='milliseconds')

fabric_projects = ['2dd1ffb8-1aff-45cc-a70d-eb93b65cc26b', '4604cab7-41ff-4c1a-a935-0ca6f20cceeb', '6b76128d-c73f-431f-a245-0397586a7d40', '32e7160e-0318-43f5-a4e3-80209f880833', 
                   '75835e68-f91f-474d-8d54-27a576cc252f', '990d8a8b-7e50-4d13-a3be-0f133ffa8653', '04b14c17-e66a-4405-98fc-d737717e2160', '1630021f-0a0c-4792-a241-997f410d36e1', 
                   '7a5adb91-c4c0-4a1c-8021-7b6c56af196f', '06e8d02a-b27f-4437-829e-8378d20e5a08', '7f33ecf0-5dd7-4fd5-b1b7-061367f8bca6']

DUMP_JSON = True

## Query Slivers

### Example: Querying Current Active Slivers on VLAN 4017

Once the Reports API client is set up, you can query for slivers with specific filters.
For example, if you want to see **all currently active slivers on VLAN 4017**, you would run a query with:

* **Sliver state** set to `"Active"` (or `"ActiveTicketed"` if you want to include slivers that are ticketed but still active).
* **VLAN** set to `4017`.

In the widget interface, this means selecting **Active** and **ActiveTicketed** under *Sliver State* and typing **4017** in the *VLAN(s)* field.

When you click **Query Slivers**, the system will send these filters to the FABRIC Reports API and return only those slivers matching the request. The results will be displayed in a Pandas DataFrame, allowing you to explore details such as:

* Project ID and name
* User associated with the sliver
* Site and host where the sliver is running
* Component type and model
* VLAN and IP subnet assignments


In [20]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
from pytz import UTC

# === Global DataFrame ===
df_slivers_filtered = pd.DataFrame()
slivers_component_types = []
sliver_types = []
slivers = []

# === Time defaults ===
default_start = datetime(2023, 6, 1, tzinfo=UTC)
default_end = datetime.now(UTC).replace(microsecond=0)

# === Helpers ===
def _csv_list(text: str):
    """Parse a comma-separated string into a clean list of strings."""
    return [t.strip() for t in text.split(",") if t.strip()]

def _csv_ints(text: str):
    """
    Parse comma-separated VLANs (ints). Supports single ints or ranges like '100-105'.
    Returns a list of ints; invalid tokens are ignored.
    """
    if not text.strip():
        return []
    out = []
    for tok in text.split(","):
        tok = tok.strip()
        if not tok:
            continue
        if "-" in tok:
            try:
                lo, hi = tok.split("-", 1)
                lo, hi = int(lo), int(hi)
                if lo <= hi:
                    out.extend(range(lo, hi + 1))
            except Exception:
                pass
        else:
            try:
                out.append(int(tok))
            except Exception:
                pass
    return out

# === Widgets ===
start_picker = widgets.DatetimePicker(description='Start:', value=default_start)
end_picker = widgets.DatetimePicker(description='End:', value=default_end)

component_type_select = widgets.SelectMultiple(
    options=["SmartNIC", "GPU", "FPGA", "Storage", "SharedNIC"],
    description="Component Type:"
)

component_model_text = widgets.Text(
    description="Component Model(s):",
    placeholder="comma-separated"
)

sliver_type_dropdown = widgets.Dropdown(
    options=[
        "VM", "Switch", "Facility", "L2STS", "L2PTP", "L2Bridge", "FABNetv4",
        "FABNetv6", "PortMirror", "L3VPN", "FABNetv4Ext", "FABNetv6Ext", "All"
    ],
    value="All",  # default = All
    description="Sliver Type:"
)

# New filters
sliver_state_select = widgets.SelectMultiple(
    options=[
        "Nascent", "Ticketed", "Active", "ActiveTicketed",
        "Closed", "CloseWait", "Failed", "Unknown", "CloseFail"
    ],
    value=["Active", "ActiveTicketed"],  # defaults
    description="Sliver State:"
)

vlan_text = widgets.Text(
    description="VLAN(s):",
    placeholder="e.g., 100, 200-205"
)

ip_subnet_text = widgets.Text(
    description="IP Subnet(s):",
    placeholder="e.g., 192.0.2.0/24, 2001:db8::/64"
)

bdf_text = widgets.Text(
    description="BDF(s):",
    placeholder="e.g., 0000:01:00.0, 0000:41:00.1"
)

host_text = widgets.Text(
    description="Host(s):",
    placeholder="hostname(s), comma-separated"
)

# New filters
user_id_text = widgets.Text(
    description="User ID(s):",
    placeholder="comma-separated"
)

user_email_text = widgets.Text(
    description="User Email(s):",
    placeholder="comma-separated"
)

project_id_text = widgets.Text(
    description="Project ID(s):",
    placeholder="comma-separated"
)

site_include_text = widgets.Text(description="Include Sites:", placeholder="comma-separated")
site_exclude_text = widgets.Text(description="Exclude Sites:", placeholder="comma-separated")

sliver_run_button = widgets.Button(description="Query Slivers")
sliver_status_output = widgets.Output()

# === Display UI ===
display(
    start_picker, end_picker,
    component_type_select, component_model_text,
    sliver_type_dropdown, sliver_state_select,
    vlan_text, ip_subnet_text, bdf_text, host_text,
    user_id_text, user_email_text, project_id_text,
    site_include_text, site_exclude_text,
    sliver_run_button, sliver_status_output
)

# === Callback ===
def run_sliver_query(b):
    global df_slivers_filtered
    global slivers
    global slivers_component_types
    global sliver_types
    sliver_status_output.clear_output()

    start_time = start_picker.value
    end_time = end_picker.value

    if not start_time or not end_time or start_time >= end_time:
        with sliver_status_output:
            print("Please ensure start time is before end time.")
        return

    component_types = list(component_type_select.value)
    slivers_component_types = component_types
    component_models = _csv_list(component_model_text.value)
    sliver_type = sliver_type_dropdown.value
    sliver_types = [sliver_type]

    sliver_states = list(sliver_state_select.value)
    vlan_list = _csv_ints(vlan_text.value)
    ip_subnets = _csv_list(ip_subnet_text.value)
    bdfs = _csv_list(bdf_text.value)
    hosts = _csv_list(host_text.value)

    user_ids = _csv_list(user_id_text.value)
    user_emails = _csv_list(user_email_text.value)
    project_ids = _csv_list(project_id_text.value)

    include_sites = _csv_list(site_include_text.value)
    exclude_sites = _csv_list(site_exclude_text.value)

    query_params = {
        "start_time": start_time.isoformat(timespec="milliseconds"),
        "end_time": end_time.isoformat(timespec="milliseconds"),
        "fetch_all": True,
        "per_page": 1000,
        "exclude_project_id": fabric_projects,  # defined elsewhere
    }

    if component_types:
        query_params["component_type"] = component_types
    if component_models:
        query_params["component_model"] = component_models
    if sliver_type != "All":
        query_params["sliver_type"] = [sliver_type]
    if include_sites:
        query_params["site_name"] = include_sites
    if exclude_sites:
        query_params["exclude_site_name"] = exclude_sites
    if sliver_states:
        query_params["sliver_state"] = sliver_states
    if vlan_list:
        query_params["vlan"] = vlan_list
    if ip_subnets:
        query_params["ip_subnet"] = ip_subnets
    if bdfs:
        query_params["bdf"] = bdfs
    if hosts:
        query_params["host"] = hosts
    if user_ids:
        query_params["user_id"] = user_ids
    if user_emails:
        query_params["user_email"] = user_emails
    if project_ids:
        query_params["project_id"] = project_ids

    sliver_run_button.disabled = True
    with sliver_status_output:
        print("Querying slivers with the following filters:")
        print(f"  Time: {start_time} → {end_time}")
        if component_types: print(f"  Component Types: {component_types}")
        if component_models: print(f"  Component Models: {component_models}")
        print(f"  Sliver Type: {sliver_type}")
        if sliver_states: print(f"  Sliver States: {sliver_states}")
        if vlan_list: print(f"  VLANs: {sorted(set(vlan_list))}")
        if ip_subnets: print(f"  IP Subnets: {ip_subnets}")
        if bdfs: print(f"  BDFs: {bdfs}")
        if hosts: print(f"  Hosts: {hosts}")
        if user_ids: print(f"  User IDs: {user_ids}")
        if user_emails: print(f"  User Emails: {user_emails}")
        if project_ids: print(f"  Project IDs: {project_ids}")
        if include_sites: print(f"  Include Sites: {include_sites}")
        if exclude_sites: print(f"  Exclude Sites: {exclude_sites}")

    try:
        response = api.query_slivers(**query_params)
        slivers = response.get("data", [])
        df_slivers_filtered = pd.DataFrame(slivers)

        sliver_status_output.clear_output()
        with sliver_status_output:
            print(f"Query complete. Total slivers: {len(df_slivers_filtered)}")
            if DUMP_JSON:
                print(json.dumps(slivers, indent=4))

        if not df_slivers_filtered.empty:
            display(df_slivers_filtered)
        else:
            with sliver_status_output:
                print("No slivers returned.")

    except Exception as e:
        with sliver_status_output:
            print(f"Error during query: {e}")

    sliver_run_button.disabled = False

# === Bind button click ===
sliver_run_button.on_click(run_sliver_query)


DatetimePicker(value=datetime.datetime(2023, 6, 1, 0, 0, tzinfo=<UTC>), description='Start:')

DatetimePicker(value=datetime.datetime(2025, 9, 24, 14, 49, 4, tzinfo=<UTC>), description='End:')

SelectMultiple(description='Component Type:', options=('SmartNIC', 'GPU', 'FPGA', 'Storage', 'SharedNIC'), val…

Text(value='', description='Component Model(s):', placeholder='comma-separated')

Dropdown(description='Sliver Type:', index=12, options=('VM', 'Switch', 'Facility', 'L2STS', 'L2PTP', 'L2Bridg…

...

Text(value='', description='Exclude Sites:', placeholder='comma-separated')

Button(description='Query Slivers', style=ButtonStyle())

Output()

,components,interfaces,ip_subnet,lease_end,lease_start,project_id,project_name,site,slice_id,sliver_id,sliver_type,state,user_email,user_id
0,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:0b.6', 'device_name...",23.134.232.64/28,2026-02-13T19:39:28+00:00,2025-08-15T19:39:29.402000+00:00,ac8d64f3-36e1-4a8e-b171-909e0e58b8c0,Extensible Internet,MAX,1e8d2575-81e7-43c7-9111-b4c46c14e204,21102769-e6e2-4516-b71a-29d9cb68ea92,fabnetv4ext,Active,scauligi@icsi.berkeley.edu,bdc77bd9-9886-4f80-ab5e-47b219e6d0cd
1,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:0c.0', 'device_name...",23.134.232.128/28,2026-02-13T19:39:28+00:00,2025-08-15T19:39:29.520000+00:00,ac8d64f3-36e1-4a8e-b171-909e0e58b8c0,Extensible Internet,UTAH,1e8d2575-81e7-43c7-9111-b4c46c14e204,d83cd977-b2c0-47f0-8ea8-7d31b05b2896,fabnetv4ext,Active,scauligi@icsi.berkeley.edu,bdc77bd9-9886-4f80-ab5e-47b219e6d0cd
2,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:06.2', 'device_name...",23.134.233.128/28,2026-02-13T19:39:28+00:00,2025-08-15T19:39:29.461000+00:00,ac8d64f3-36e1-4a8e-b171-909e0e58b8c0,Extensible Internet,INDI,1e8d2575-81e7-43c7-9111-b4c46c14e204,6afd309b-ac79-4b58-812b-4295a75e66cf,fabnetv4ext,Active,scauligi@icsi.berkeley.edu,bdc77bd9-9886-4f80-ab5e-47b219e6d0cd
3,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:0a.0', 'device_name...",23.134.232.48/28,2026-02-13T20:14:33+00:00,2025-08-15T20:14:33.304000+00:00,ac8d64f3-36e1-4a8e-b171-909e0e58b8c0,Extensible Internet,STAR,98066e30-2ae5-4f53-b633-ba33c7def920,ad485ef7-510d-4c19-8042-a83ff61d06f6,fabnetv4ext,Active,scauligi@icsi.berkeley.edu,bdc77bd9-9886-4f80-ab5e-47b219e6d0cd
4,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:a1:0a.1', 'device_name...",23.134.232.48/28,2026-01-31T16:33:28+00:00,2025-09-15T15:33:28.594000+00:00,c768a8b8-a19b-4366-be8c-e735dcccb027,NSF CI Compass,STAR,82b581d4-0f6b-4b7d-abd0-b7957a15d937,6f19c948-5147-414e-b112-a03ff19750f2,fabnetv4ext,Active,kthare10@email.unc.edu,43b7271b-90eb-45f6-833a-e51cf13bbc68
5,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:00.6', 'device_name...",23.134.232.48/28,2025-10-06T14:20:11+00:00,2025-09-22T14:20:11.625000+00:00,c13c119f-63ea-42f8-a813-c90ee6c580de,FABRIC-OOD,STAR,6a8b9175-d463-4a4c-9fd4-fee8c8e9401c,7a8ad88a-5a8c-4f31-810f-76526f23ade8,fabnetv4ext,Active,pinyi.shi@uky.edu,3fbc9d2a-0ccb-4c0d-9670-f839ff4486d6
6,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:01.5', 'device_name...",23.134.232.80/28,2025-10-06T14:20:11+00:00,2025-09-22T14:20:11.572000+00:00,c13c119f-63ea-42f8-a813-c90ee6c580de,FABRIC-OOD,TACC,6a8b9175-d463-4a4c-9fd4-fee8c8e9401c,99c8b038-6c0c-4afc-b727-be96cb071401,fabnetv4ext,Active,pinyi.shi@uky.edu,3fbc9d2a-0ccb-4c0d-9670-f839ff4486d6
7,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:08.3', 'device_name...",23.134.232.208/28,2025-10-06T14:20:11+00:00,2025-09-22T14:20:11.610000+00:00,c13c119f-63ea-42f8-a813-c90ee6c580de,FABRIC-OOD,UCSD,6a8b9175-d463-4a4c-9fd4-fee8c8e9401c,c10fca85-2ac2-4241-bbc0-89f4e8e6f66a,fabnetv4ext,Active,pinyi.shi@uky.edu,3fbc9d2a-0ccb-4c0d-9670-f839ff4486d6
8,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:03.4', 'device_name...",23.134.232.208/28,2025-09-29T14:57:15+00:00,2025-09-08T14:57:15.839000+00:00,b72a2eb9-2f8c-4c19-8661-210f07efba12,FAB-cache,UCSD,ad816088-74bd-4893-97e6-11d166eaccb3,4c373457-ca10-4169-8990-0d64201cbaa2,fabnetv4ext,Active,pinyi.shi@uky.edu,3fbc9d2a-0ccb-4c0d-9670-f839ff4486d6
9,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:e2:06.7', 'device_name...",23.134.232.160/28,2025-10-04T17:56:10+00:00,2025-09-19T17:56:10.868000+00:00,8e0099fa-e38f-4020-9959-4289039d36bd,Ecosystem for Research Networking,WASH,d7fd3d00-5b45-4e5e-a398-4d5199376710,ecea8d6f-6c13-4e02-9902-53cfa164c9bc,fabnetv4ext,Active,maureen.c.dougherty@rutgers.edu,e9db6486-f01a-4284-a3c1-335b17c3a825


,components,interfaces,ip_subnet,lease_end,lease_start,project_id,project_name,site,slice_id,sliver_id,sliver_type,state,user_email,user_id
0,"{'data': [], 'total': 0}","{'data': [{'bdf': '0000:a1:0a.1', 'device_name...",23.134.232.48/28,2026-01-31T16:33:28+00:00,2025-09-15T15:33:28.594000+00:00,c768a8b8-a19b-4366-be8c-e735dcccb027,NSF CI Compass,STAR,82b581d4-0f6b-4b7d-abd0-b7957a15d937,6f19c948-5147-414e-b112-a03ff19750f2,fabnetv4ext,Active,kthare10@email.unc.edu,43b7271b-90eb-45f6-833a-e51cf13bbc68
